In [ ]:
import pickle
import time

import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict

from old_gep_code.primal_dual import PrimalDualTrainer
from old_gep_code.gep_problem import GEPProblem
from gep_main import run_model as run_Gurobi
from gep_main import run_model_no_bounds as run_Gurobi_no_bounds
from get_gurobi_vars import OptValueExtractor
import sys
import torch

import pyomo as pyo

Running on cpu
Parsing the config file
Initializing the solver
Using Gurobi


In [2]:
DEVICE = 'cpu'
args = {
        "K": 10,
        "L": 500,
        "tau": 0.8,
        "rho": 5,
        "rho_max": 50000,
        "alpha": 100,
        "batch_size": 200,
        "hidden_sizes": [500, 500],
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

# filepath = "reproduction-experiments/datasets/simple/random_simple_dataset_var100_ineq50_eq50_ex10000"
# filepath = "CUSTOM_random_simple_dataset_var100_ineq50_eq50_ex1000"
filepath = "reproduction-experiments/datasets/simple/random_simple_dataset_var100_ineq50_eq50_ex10000"

with open(filepath, 'rb') as f:
    data = pickle.load(f)
# for attr in dir(data):
#     var = getattr(data, attr)
#     if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
#         try:
#             setattr(data, attr, var.to(dtype=torch.float64, device=DEVICE))
#         except AttributeError:
#             pass
data._device = DEVICE
run_name = "original_problem"
save_dir = os.path.join('outputs', 'PDL_benchmark',
    run_name + "-" + str(time.time()).replace('.', '-'))

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
    pickle.dump(args, f)

# Run PDL
trainer = PrimalDualTrainer(data, args, save_dir, problem_type="Benchmark", log=True)
primal_net, dual_net, stats = trainer.train_PDL()

X dim: 50
Y dim: 100
Size of mu: 50
Size of lambda: 50


KeyboardInterrupt: 

In [ ]:
opt_Y = data.calc_Y()
print(opt_Y.shape)
opt_obj_values = data.obj_fn(torch.tensor(opt_Y))

predicted_Y = primal_net(data.trainX)
pred_obj_values = data.obj_fn(predicted_Y)

print(opt_obj_values[:83].mean().item(), pred_obj_values.mean().item())

running osqp
(100, 100)
-13.025893973281029 -12.851622816816866


In [ ]:
predicted_Y = primal_net(data.validX)

obj_values = data.obj_fn(predicted_Y)

print(obj_values)

tensor([  5.5901,  10.6828,  -5.3187,   5.4430,   7.2591,  -4.8035, -11.0495,
         12.1373], grad_fn=<SumBackward1>)
